# Classification of ASL Hand Signs 

Group 2 - Hui Hua (Emily) Huang, Jasper Precilla

## Importing Libraries

In [13]:
#need to run so that cv2 can be imported 
!pip install opencv-python

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
#import statements
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
import glob
import xml.etree.ElementTree as ET
from PIL import Image
import os
import shutil
import random

In [2]:
#installing ultralytics
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.27 🚀 Python-3.10.0 torch-2.2.1+cpu CPU (AMD Ryzen 7 4800HS with Radeon Graphics)
Setup complete ✅ (16 CPUs, 15.4 GB RAM, 453.5/457.9 GB disk)


In [5]:
#importing YOLO model
from ultralytics import YOLO

## Data Preprocessing

In [8]:
#defining data sets
asl_dataset = 'full_alphabet_data'
asl_test = 'full_alphabet_data/asl_alphabet_test'
asl_train = 'full_alphabet_data/asl_alphabet_train'

Because our dataset has 9010 images split across 3 classes for the training set and only 3 images split across 3 classes for the testing set, we are rearranging the dataset and pulling a number of images from the training into the testing set to achieve a 80:20 train:test ratio. 

In [18]:
#splitting images into an 80/20 ratio
#total images 9000 + 3 + 60 = 9073 -> 80%: 7258 20%: 
classes = ['A','B','C']
src = '/Users/jaspe/Documents/School/IAT481/asl_alphabet_a_to_c/src'
train = '/Users/jaspe/Documents/School/IAT481/asl_alphabet_a_to_c/train'
test = '/Users/jaspe/Documents/School/IAT481/asl_alphabet_a_to_c/test'

def splitimg_80_20():
    for letter in classes:
        src_folder = os.path.join(src, letter)
        src_imgs = glob.glob(src_folder+'/*.jpg')
        count=1

        #RANDOMLY TAKE 600/3000 
        random.shuffle(src_imgs)
        while count <= 600:
            src_path = src_imgs[count]
            dst_path = test + '/' + letter +'/' + letter + str(count) + '.jpg'
            shutil.move(src_path, dst_path)
            count = count+1

# FOR FIXING LABELS AND MOVING REMAINING IMAGES TO TRAIN FOLDERS
def renumber_imgs():
    for letter in classes:
        src_folder = os.path.join(src, letter)
        src_imgs = glob.glob(src_folder+'/*.jpg')
        iterator = 1;
        for img in src_imgs:
            # print(train+'/'+letter+'/'+letter+str(iterator)+'.jpg')
            os.rename(img, train+'/'+letter+'/'+letter+str(iterator)+'.jpg')
            iterator = iterator+1

## Training YOLO Model

In [10]:
#loading pre-trained model
asl_model = YOLO('yolov8n-cls.pt')
# !ls runs/classify/train/weights

In [11]:
#train on dataset 
asl_results = asl_model.train(data=asl_dataset, epochs=2, batch=24)

New https://pypi.org/project/ultralytics/8.2.0 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.27 🚀 Python-3.10.0 torch-2.2.1+cpu CPU (AMD Ryzen 7 4800HS with Radeon Graphics)
engine\trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=full_alphabet_data, epochs=2, time=None, patience=100, batch=24, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fa

RuntimeError: Dataset 'full_alphabet_data' error  [WinError 3] The system cannot find the path specified: 'C:\\Users\\jaspe\\Documents\\GitHub\\iat-481-final-proj\\full_alphabet_data\\train'

In [25]:
#validate data and get metrics/results
asl_model = YOLO('runs/classify/train72/weights/best.pt')
asl_metrics = asl_model.val()  
asl_metrics.top1  
asl_metrics.top5  



Ultralytics YOLOv8.1.27 🚀 Python-3.10.0 torch-2.2.1+cpu CPU (AMD Ryzen 7 4800HS with Radeon Graphics)
YOLOv8n-cls summary (fused): 73 layers, 1438723 parameters, 0 gradients, 3.3 GFLOPs
train: C:\Users\jaspe\Documents\GitHub\iat481-computer-vision\data\train... found 7242 images in 3 classes ✅ 
val: C:\Users\jaspe\Documents\GitHub\iat481-computer-vision\data\val... found 918 images in 3 classes ✅ 
test: C:\Users\jaspe\Documents\GitHub\iat481-computer-vision\data\test... found 900 images in 3 classes ✅ 


val: Scanning C:\Users\jaspe\Documents\GitHub\iat481-computer-vision\data\val... 918 images, 0 corrupt: 100%|██████████| 918/918 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 58/58 [00:05<00:00,  9.69it/s]


                   all          1          1
Speed: 0.0ms preprocess, 4.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\val12


[]

In [ ]:
#get results
results = asl_model('maxresdefault.jpg')

for r in results:   
    print(r.probs)  # print the Probs object containing the detected class probabilities


image 1/1 c:\Users\jaspe\Documents\GitHub\iat481-computer-vision\maxresdefault.jpg: 224x224 B 0.64, A 0.23, C 0.13, 18.1ms
Speed: 10.6ms preprocess, 18.1ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
ultralytics.engine.results.Probs object with attributes:

data: tensor([0.2295, 0.6443, 0.1262])
orig_shape: None
shape: torch.Size([3])
top1: 1
top1conf: tensor(0.6443)
top5: [1, 0, 2]
top5conf: tensor([0.6443, 0.2295, 0.1262])


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ae844523-e20e-4ba0-a57a-18d4a32afd51' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>